# B - End-to-End ML Part 1: Feature Engineering + Baseline Modell

Environment für dieses Notebook: `ads-ml-full` (*requirements-py3.11-ads-ml-full.txt*).


## Daten einlesen

In [1]:
from repml.datasets.trees_ber import read_trees_ber

data = read_trees_ber()

In [2]:
data.sample(5, random_state=42)

,baumid,standortnr,kennzeich,namenr,art_dtsch,art_bot,gattung_deutsch,gattung,stammumfg,bezirk,eigentuemer,pflanzjahr,standalter,kronedurch,baumhoehe,lon,lat
235708,00008100:00154635,19,228600,Zwinglistr. 18-19,Silber-Ahorn,Acer saccharinum,AHORN,ACER,149.0,Mitte,Land Berlin,1950.0,73.0,15.0,14.0,13.329312,52.526292
314780,00008100:001380be,20,00481,Carl-Sonnenschein Grundschule 31.G,Hainbuche,Carpinus betulus,HAINBUCHE,CARPINUS,193.0,Tempelhof-Schöneberg,Land Berlin,1964.0,59.0,NaN,12.0,13.407388,52.443216
50131,00008100:000ef4f9,011415,101491,Schweizerhof Park,Buche,Fagus spec.,BUCHE,FAGUS,100.0,Steglitz-Zehlendorf,Land Berlin,NaN,NaN,NaN,NaN,13.259721,52.422273
256683,00008100:0028c98c,A/364,01019,Peter-Witte-Park,"Süss-Kirsche,Vogel-Kirsche",Prunus avium,NaN,PRUNUS,141.0,Reinickendorf,Land Berlin,NaN,NaN,NaN,20.0,13.325103,52.587966
100323,00008100:001fa47b,17,28GA05,Sewanstr. / Schwimmhalle,Spätblühende Trauben-Kirsche,Prunus serotina,NaN,PRUNUS,85.0,Lichtenberg,Land Berlin,NaN,NaN,5.0,7.0,13.515136,52.495615


### Outlier & Co

Auch Outliers können wir mit Fachwissen begegnen, bevor wir robuste Transformationen anwenden.

Bäume 141 Meter dick?

In [3]:
data["stammumfg"].sort_values(ascending=False).iloc[:5]

373198    97106.0
78518     92100.0
292637    44425.0
303346    28029.0
303299    25827.0
Name: stammumfg, dtype: float64

Annahme: 5 Meter reicht.

In [4]:
import numpy as np

data.loc[data["stammumfg"] > 1665, "stammumfg"] = np.nan

Kronendurchmesser von 6,8 KM?

In [5]:
data["kronedurch"].sort_values(ascending=False).iloc[:5]

334466    6800.0
172478    3600.0
388911    3158.0
390355    1800.0
140705    1800.0
Name: kronedurch, dtype: float64

70 Meter wäre schon sehr groß.

In [6]:
data.loc[data["kronedurch"] > 70, "kronedurch"] = np.nan

Baumhöhe von 2800 Meter?

In [7]:
data["baumhoehe"].sort_values(ascending=False).iloc[:5]

176186    25700.0
176215    25700.0
210317    12700.0
210022     5600.0
100243     3000.0
Name: baumhoehe, dtype: float64

100 schafft auch keiner.

In [8]:
data.loc[data["baumhoehe"] >= 100, "baumhoehe"] = np.nan

## Bäume aus dem Jahre 0 und der Zukunft?

In [9]:
data["pflanzjahr"].min()

0.0

In [10]:
data["pflanzjahr"].max()

88888888888.0

In [11]:
len(data[(data["pflanzjahr"] < 1400) | (data["pflanzjahr"] > 2023)])

61

In [12]:
data.loc[(data["pflanzjahr"] < 1400) | (data["pflanzjahr"] > 2023), "pflanzjahr"] = np.nan

## Feature Engineering

### Features ausschließen?

Mit fachlichen Wissen können wir Features ausschließen ❌ oder beibehalten ✅, unabhängig von der detaillierten Untersuchung der Werte. Insbesondere wollen wir nicht keine (zusammengesetzen) IDs für Einzelwerte dabei haben.


- ❌ `baumid`: Wenn auch nicht perfekt eindeutig, hier ist das Ziel ganz klar einzelne Bäume zu identifizieren.
- ❌ `standortnr`: Lt. Datenbeschreibung die ID auf bezirklicher Ebene. Zusammen mit dem Bezirk lässt sich wieder auf den einzelnen Baum schließen!
- ❌ `standalter`: Scheint berechnet zu sein, daher duplizierte Info!

- ✅ `kennzeich`: Lt. Datenbeschreibung die bezirkliche Nummer des zugeordneten Pflegeobjektes. Ein Pflegeobjekt kann viele Bäume haben, nur bei geringen Anzahlen gilt Acht!
- ✅ `namenr`: Hier gilt Gleiches wie bei `kennzeich`, bloß dass es sich um den Namen und nicht die Nummer handelt.

In [13]:
del data["baumid"]
del data["standortnr"]
del data["standalter"]

### Feature aus der `namenr` berechnen?

In [14]:
data["namenr"].value_counts().index[350:400]

CategoricalIndex(['Fernsehturmanlage zw. Fernsehturm u. Spandauer Str.',
                  'Urnenhain', 'Hatzfeldtallee 19+31 (Sport)',
                  'Ortelsburgpark', 'Stolzenfelsstr. / entlang der Reichsbahn',
                  'Grünanlage Cleantech-Business-Park II', 'Rheinsteinpark',
                  'Langhoffstr. 2 -26/ Murtzaner Ring 34-68',
                  'Gemeindepark Lankwitz - Teil 1/2', 'Lietzenseepark/Süd',
                  'Gutspark am Loeperplatz / Möllendorfsstr. 34-39',
                  'Im Fischgrund, "Rosenanger"', 'Marx-Engels-Forum',
                  'Brusebergstr., Grünzug Lärchenkamp',
                  'Altenhofer Dreieck / GA', 'Schloßpark Lichterfelde',
                  'Jochen-Klepper-Weg WG - Teil 1/2', 'Harbigstr. 40',
                  'Havelpromenade- Süd', 'Schönagelstraße/ Blumberger Damm',
                  'Friedhof-Staaken', 'Monbijou Park',
                  'Hahneberg-GA Hänge und Weiden',
                  'Lauterbach-Grundschule (33. G

In [15]:
data["namenr"].nunique()

4688

Regex ersetzt Zahlen (arabische wie römische) und +- zwischen den Zahlen.

In [16]:
import re

data["namenr_nonum"] = data["namenr"].apply(
    lambda key: re.sub(r"\d+(-\d+)?|\b[IVXLCDM]+\b", "", key).strip()
)

In [17]:
data["namenr_nonum"].nunique()

4407

In [18]:
data["namenr_nonum"].value_counts().index[350:400]

Index(['Friedhof Wannsee - /', 'Havelchaussee  - Stößenseebrücke - GA',
       'Harbigstr.', 'Hatzfeldtallee + (Sport)',
       'Stolzenfelsstr. / entlang der Reichsbahn', 'Ortelsburgpark',
       'Grünanlage Cleantech-Business-Park', 'Langhoffstr.  -/ Murtzaner Ring',
       'Rheinsteinpark', 'Bahnhofstr.', 'Lietzenseepark/Süd',
       'Im Fischgrund, "Rosenanger"',
       'Gutspark am Loeperplatz / Möllendorfsstr.', 'Marx-Engels-Forum',
       'Kleistgrab WG - Teil /', 'Brusebergstr., Grünzug Lärchenkamp',
       'Altenhofer Dreieck / GA', 'Schloßpark Lichterfelde',
       'Schönagelstraße/ Blumberger Damm', 'Havelpromenade- Süd',
       'Hahneberg-GA Hänge und Weiden', 'Monbijou Park', 'Friedhof-Staaken',
       'John-F.-Kennedy-Schule F + SP', 'Lauterbach-Grundschule (. G)',
       'Albert-Einstein-Oberschule / Filiale Alfred-Nobel-Sekundarschule',
       'Bremer Str.', 'Matthias-Claudius-Grundschule',
       'Niederneuendorfer Allee- Buskehre', 'Stade de Napoleon', 'Haveldüne',
  

### Georaster

In [21]:
data["lon"].sample(5)

326205    13.432796
281751    13.157988
188000    13.471386
69180     13.362830
382135    13.571441
Name: lon, dtype: float64

In [22]:
data["lon_section"] = data["lon"].round(1).astype("category")
data["lat_section"] = data["lat"].round(1).astype("category")

In [24]:
data["lon_section"].cat.categories

Index([13.1, 13.2, 13.3, 13.4, 13.5, 13.6, 13.7], dtype='float64')

### Ratios

2 neue Features: Baumhöhe zum Kronendurchmesser und Baumhöhe zum Stammumfang.

In [25]:
data["hoehe_zu_krone"] = data["baumhoehe"] / data["kronedurch"].replace(0, np.nan)
data["hoehe_zu_stamm"] = data["baumhoehe"] / data["stammumfg"].replace(0, np.nan)

### Baumart - Laubbaum oder Nadelbaum?

Mapping Tabelle mit ChatGPT & Google erstellt.

In [26]:
data["gattung"] = data["gattung"].cat.rename_categories({"Abies": "ABIES"})

In [27]:
import pandas as pd

genus_cat_mapping = pd.read_parquet("../data/berlin_genus_category_mapping.parquet")
genus_cat_mapping

,gattung,baumart
0,ABIES,Nadelbaum
1,ACER,Laubbaum
2,AESCULUS,Laubbaum
3,AILANTHUS,Laubbaum
4,ALNUS,Laubbaum
...,...,...
93,TSUGA,Nadelbaum
94,ULMUS,Laubbaum
95,UNBEKANNT,UNBEKANNT
96,WISTERIA,Laubbaum


In [28]:
data = data.merge(genus_cat_mapping, on="gattung", how="left")

In [29]:
data[["gattung_deutsch", "baumart"]].value_counts()[:10]

gattung_deutsch  baumart  
AHORN            Laubbaum     103851
EICHE            Laubbaum      50781
LINDE            Laubbaum      38056
ROBINIE          Laubbaum      23080
HAINBUCHE        Laubbaum      20894
BIRKE            Laubbaum      20712
PAPPEL           Laubbaum      17905
KIEFER           Nadelbaum     17359
BUCHE            Laubbaum      14130
ESCHE            Laubbaum      11536
Name: count, dtype: int64

In [30]:
data["gattung_deutsch"] = data["gattung_deutsch"].cat.add_categories(["unknown"])
data["gattung_deutsch"] = data["gattung_deutsch"].fillna("unknown")

### Flag für infrequente Kategorien (< 1000 Member)

In [31]:
replacement_cols = ["art_dtsch", "art_bot", "gattung_deutsch", "gattung"]

threshold = 1000

for col in replacement_cols:
    data[col + "_infrequent"] = data[col].map(data[col].value_counts() < threshold)

In [32]:
data[[col + "_infrequent" for col in replacement_cols]].sample(5, random_state=42)

,art_dtsch_infrequent,art_bot_infrequent,gattung_deutsch_infrequent,gattung_infrequent
235708,False,False,False,False
314780,False,False,False,False
50131,False,False,False,False
256683,False,False,False,False
100323,True,True,False,False


## Labeled & unlabeled data
Es fehlen viele Datenpunkte für "Pflanzjahr", wir teilen das Datenset entlang dieses Merkmals.

In [33]:
unlabeled = data[data["pflanzjahr"].isna()].copy()
len(unlabeled)

141725

In [34]:
labeled = data[data["pflanzjahr"].notna()]
len(labeled)

275391

## Rare Kategorien ersetzen

Erst möglich nach dem Split!

In [35]:
labeled = labeled.copy()
minimum_count_threshold = 20

cat_cols = ["bezirk", "art_dtsch", "art_bot", "gattung_deutsch", "gattung", "kennzeich", "namenr"]

for cat_col in cat_cols:
    counts = labeled[cat_col].value_counts()
    rare = counts[counts < minimum_count_threshold].index.to_list()

    if len(rare) > 0:
        print(f"In '{cat_col}' werden {len(rare)} seltene Kategorien ersetzt!")
        labeled[cat_col] = labeled[cat_col].astype("object")
        labeled.loc[labeled.query(f"{cat_col} == @rare").index, cat_col] = "rare"
        labeled[cat_col] = labeled[cat_col].astype("category")
        labeled[cat_col] = labeled[cat_col].cat.remove_unused_categories()

In 'art_dtsch' werden 455 seltene Kategorien ersetzt!
In 'art_bot' werden 484 seltene Kategorien ersetzt!
In 'gattung_deutsch' werden 29 seltene Kategorien ersetzt!
In 'gattung' werden 31 seltene Kategorien ersetzt!
In 'kennzeich' werden 2467 seltene Kategorien ersetzt!
In 'namenr' werden 2779 seltene Kategorien ersetzt!


## Trainings- und Testdaten

In [37]:
from sklearn.model_selection import train_test_split

y = "pflanzjahr"
X = labeled.columns.to_list()
X.remove(y)
train_data, test_data = train_test_split(
    labeled,
    test_size=0.2,
    random_state=42,
    stratify=labeled["gattung_deutsch"],
)

## Pipeline Design & Modellauswahl

In [38]:
num_features = ["kronedurch", "stammumfg", "baumhoehe", "hoehe_zu_krone", "hoehe_zu_stamm"]

### Versuch 1: Simple Linear Regression

In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

ct_1 = ColumnTransformer(
    transformers=[
        ("imp", SimpleImputer(), num_features),
    ],
)

In [40]:
from sklearn.linear_model import LinearRegression

est_1 = LinearRegression()

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipeline_1 = Pipeline(steps=[("ct1", ct_1), ("scale", StandardScaler()), ("model_linreg", est_1)])

In [42]:
pipeline_1.fit(X=train_data[num_features], y=train_data[y])

Pipeline(steps=[('ct1',
                 ColumnTransformer(transformers=[('imp', SimpleImputer(),
                                                  ['kronedurch', 'stammumfg',
                                                   'baumhoehe',
                                                   'hoehe_zu_krone',
                                                   'hoehe_zu_stamm'])])),
                ('scale', StandardScaler()),
                ('model_linreg', LinearRegression())])

In [43]:
pipeline_1.score(X=test_data[num_features], y=test_data[y])

0.520027700755986

In [44]:
from sklearn.model_selection import cross_val_score

cross_val_score(estimator=pipeline_1, X=train_data[num_features], y=train_data[y], cv=5)

array([0.51420325, 0.53161126, 0.52440537, 0.52879704, 0.51962954])